In [1]:
### Imports
import os
import sys
sys.path.append('C:\\Users\\szb37\\My Drive\\Projects\\Unmasking power\\UP codebase')
from statsmodels.stats.proportion import proportion_confint
import src.folders as folders
import src.power as power
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 300  # Set display DPI
plt.rcParams.update({'font.family': 'arial'})
title_fontdict = {'fontsize': 16, 'fontweight': 'bold'}
axislabel_fontdict = {'fontsize': 14, 'fontweight': 'bold'}
ticklabel_fontsize = 12

dir_out=os.path.join(folders.exports, 'Bayesian')
def save_fig(fig, fname, dir_out=dir_out):
    for fileformat in ['png', 'svg']:
        fig.savefig(
            fname=os.path.join(dir_out, f'{fname}.{fileformat}'),
            bbox_inches='tight',
            format=fileformat,
            dpi=300,)

n_draws = 300
n_trials = 200
sample = 200

In [2]:
### Generate mock data
# We will sample this df; precompute for convenience


# df_patientsData => range of CGRs
cgrs = [round(float(cgr), 2) for cgr in np.arange(0, 1.01, 0.02)]
df_patientsData=[]
for cgr in cgrs:
    df_patientsData.append(
        power.DataGeneration.get_df_patientsData(
            scenario = f'{cgr}', 
            n_trials = n_trials, 
            sample = sample, 
            params = [{
                'type': 'binaryguess',
                'arm_params':{
                    'C': {'cgr': cgr},
                    'T': {'cgr': cgr},},}]))       
df_patientsData = pd.concat(df_patientsData, ignore_index=True)
df_patientsData['cgr'] = pd.to_numeric(df_patientsData['scenario'], errors='coerce')
del df_patientsData['scenario']
df_patientsData = df_patientsData[['cgr','trial','pID','trt','guess_bin',]]
# df_patientsData.head()

Define functions

In [3]:
### get_df_bayes(): df of the prior, likelihood and posterior distributions
def get_df_bayes(df_patientsData, n_obs, n_match=None, target_ci=None, norm='sum',  trial_name='tmp', n_draws=n_draws, df_bayes=None):

    assert isinstance (df_patientsData, pd.DataFrame)

    assert norm in ['sum', 'trapz']
    assert (isinstance(trial_name, str) or (trial_name is None))
    assert (isinstance(df_bayes, pd.DataFrame) or (df_bayes is None))
    assert isinstance (n_draws, int)
    assert isinstance (n_obs, int)

    # Target CI is either supplied by user or calculated from n_match and n_obs
    if target_ci is None:
        assert isinstance (n_match, int)
        target_ci = proportion_confint(n_match, n_obs, method='jeffreys')

    target_ciL = target_ci[0]
    target_ciH = target_ci[1]

    cgrs = df_patientsData.cgr.unique().tolist()
    trials = df_patientsData.trial.unique().tolist()    

    # Use flat prior if other prior is not supplied
    if df_bayes is None: 
        df_bayes = pd.DataFrame()
        df_bayes['cgr'] = cgrs
        df_bayes['prior'] = 1 / len(cgrs)

    ### Sample from the df_patientsdata to get lklhoods
    rows = []

    if trial_name is not None:
        loop_desc=f'Calc lklhoods for {trial_name}'
    else:
        loop_desc='Calc lklhoods'

    for cgr in tqdm(cgrs, desc=loop_desc):
        for trial in trials:
            
            df_tmp = df_patientsData.loc[
                (df_patientsData.cgr == cgr) & 
                (df_patientsData.trial == trial)]
            
            if len(df_tmp) == 0:
                continue
            
            if len(df_tmp) < n_obs:
                warnings.warn(f'Not enough data at CGR={cgr}, trial={trial} for sampling')
                continue


            # Precompute match array for all rows in this group
            match_arr = (df_tmp['trt'] == df_tmp['guess_bin']).values.astype(float)
            n_pool = len(match_arr)
            
            # Generate all random indices at once: (n_draws, n_obs)
            idx = np.random.randint(0, n_pool, size=(n_draws, n_obs))
            
            # Compute CGR for all n_draws at once
            cgr_samples = match_arr[idx].mean(axis=1)
            
            # Count how many fall in range
            n_inrange = ((cgr_samples >= target_ciL) & (cgr_samples <= target_ciH)).sum()
            
            row = {}
            row['cgr'] = cgr
            row['trial'] = trial
            # This is P(sample CGR ∈ target CI | true CGR)
            row['lklhood'] = round(n_inrange / n_draws, 4)  
            rows.append(row)

    # Calculate average liklihood across trials
    df_likelihood = pd.DataFrame(rows)
    df_lklhood = df_likelihood.groupby('cgr')['lklhood'].mean().reset_index()
    df_lklhood.columns = ['cgr', 'lklhood']
    df_bayes = df_bayes.merge(df_lklhood, on='cgr', how='inner')

    # Calculate posterior: prior * likelihood, then normalize
    df_bayes['posterior'] = df_bayes['prior'] * df_bayes['lklhood']

    if norm=='sum':
        df_bayes['posterior'] = df_bayes['posterior'] / df_bayes['posterior'].sum()
    elif norm=='trapz':
        auc = np.trapezoid(y=df_bayes['posterior'], x=df_bayes['cgr'])
        df_bayes['posterior'] = df_bayes['posterior'] / auc

    df_bayes['posterior_cum'] = df_bayes['posterior'].cumsum()

    if trial_name is not None:
        df_bayes.insert(0, 'trial', trial_name)
    
    return df_bayes

In [4]:
### get_df_posterior_sum(): sum of the posterior in a given range
cgr_ranges = [
    (0.0, 0.375),
    (0.375, 0.625),
    (0.625, 1.0)]

def get_df_posterior_sum(df_bayes, cgr_ranges=cgr_ranges, wide=True, digits=3):

    assert isinstance(df_bayes, pd.DataFrame)
    assert isinstance(digits, int)

    rows = []
    for trial in df_bayes.trial.unique():
        df_trial = df_bayes.loc[df_bayes.trial == trial]
        for cgr_range in cgr_ranges:

            row={}
            row['trial'] = trial
            row['cgr_range'] = cgr_range
            row['posterior_sum'] = round(df_trial.loc[
                ((df_trial.cgr > cgr_range[0]) & (df_trial.cgr <= cgr_range[1])), 
                'posterior'].sum(), digits)
            rows.append(row)

    df_posterior_sum = pd.DataFrame(rows)

    if wide:
        df_posterior_sum = df_posterior_sum.pivot(
            index='trial', 
            columns='cgr_range', 
            values='posterior_sum')
        df_posterior_sum.reset_index(inplace=True)
        df_posterior_sum.columns.name=None

    
    return df_posterior_sum

In [5]:
### draw_posterior(): draw (cumulative) posterior distribution
def draw_posterior(df_bayes, cumulative=False, save=True): 

    assert isinstance(df_bayes, pd.DataFrame)
    assert isinstance(cumulative, bool)
    assert isinstance(save, bool)
    
    for trial in df_bayes.trial.unique():

        fig, ax = plt.subplots()

        if cumulative:
            sns.lineplot(
                data = df_bayes.loc[(df_bayes.trial==trial)],
                x = 'cgr', 
                y = 'posterior_cum',)
            ax.set_ylabel('Cumulative p(CGR | data)', fontsize=14, fontweight='bold')
            fname = f'posterior_cumulative_{trial}'
        else:
            sns.lineplot(
                data = df_bayes.loc[(df_bayes.trial==trial)],
                x = 'cgr', 
                y = 'posterior',)
            ax.set_ylabel('p(CGR | data)',fontsize=14, fontweight='bold')
            fname = f'posterior_{trial}'
        
        plt.title(f'{trial} trial', fontweight='bold', fontsize=18)
        ax.set_xlabel('CGR', fontsize=14, fontweight='bold')
        ax.set_ylim([-0.003, 0.1])

        if save:
            save_fig(fig, fname)

        plt.show()

Calculate df_bayes of various studies

In [ ]:
### Calculate df_bayes for trials

df_bayes =[]

df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'Avery',
        n_match = 32,
        n_obs = 63,))

df_bayes.append(
    get_df_bayes(
    df_patientsData = df_patientsData, 
    trial_name= 'Jelovac',
    n_match = 38,
    n_obs = 60,))

df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'Richards',
        n_match = 57,
        n_obs = 97,))

df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'Sloshower',
        n_match = 27,
        n_obs = 34,))

# df_bayes = get_df_bayes(
#     df_patientsData = df_patientsData, 
#     trial_name= 'Lewis',
#     n_match = 348,
#     n_obs = 552,)

# df_bayes = get_df_bayes(
#     df_patientsData = df_patientsData, 
#     trial_name= 'POP',
#     n_match = 20,
#     n_obs = 24,)

df_bayes = pd.concat(df_bayes)

In [ ]:
### Save / load df_bayes
#df_bayes.to_csv(os.path.join(dir_out, 'df_bayes.csv'), index=False)
df_bayes = pd.read_csv(os.path.join(dir_out, 'df_bayes.csv'))

Run analysis

In [ ]:
### Calc posterior sum in ranges of interest 
df_posterior_sum = get_df_posterior_sum(df_bayes)
df_posterior_sum.to_csv(os.path.join(dir_out, 'df_posterior_sum.csv'), index=False)
df_posterior_sum

In [ ]:
### Draw posteriors
draw_posterior(df_bayes, cumulative=False)
draw_posterior(df_bayes, cumulative=True)

In [ ]:
### Calc Bayes' factors

for trial in df_bayes.trial.unique():
    df_trial = df_bayes.loc[df_bayes.trial == trial]

    h1_mask = (df_trial.cgr > 0.375) & (df_trial.cgr <= 0.625)
    h0_mask = (df_trial.cgr >  0.625) & (df_trial.cgr <= 1)

    # Marginal likelihood = sum(prior * likelihood) over each range
    ml_h1 = (df_trial.loc[h1_mask, 'prior'] * df_trial.loc[h1_mask, 'lklhood']).sum()
    ml_h0 = (df_trial.loc[h0_mask, 'prior'] * df_trial.loc[h0_mask, 'lklhood']).sum()

    bf = ml_h1 / ml_h0
    print(f'Bayes factor for {trial}: {round(bf, 3)}')

# Experiments

In [6]:
### Impact of CI and sample size

# mean CGR:0.7
# small CI: (0.65, 0.75)
# large CI: (0.55, 0.85)
# small sample: 50
# large sample: 150

df_bayes =[]
df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'smallCI_smallSample',
        n_obs = 50,
        target_ci = (0.65, 0.75)))
df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'smallCI_largeSample',
        n_obs = 150,
        target_ci = (0.65, 0.75)))
df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'largeCI_smallSample',
        n_obs = 50,
        target_ci = (0.55, 0.85)))
df_bayes.append(
    get_df_bayes(
        df_patientsData = df_patientsData, 
        trial_name= 'largeCI_largeSample',
        n_obs = 150,
        target_ci = (0.55, 0.85)))

df_bayes = pd.concat(df_bayes)
get_df_posterior_sum(df_bayes)

Calc lklhoods for largeCI_largeSample: 100%|██████████| 51/51 [02:46<00:00,  3.26s/it]


,trial,"(0.0, 0.375)","(0.375, 0.625)","(0.625, 1.0)"
0,largeCI_largeSample,0.000,0.276,0.724
1,largeCI_smallSample,0.001,0.295,0.704
2,smallCI_largeSample,0.000,0.127,0.873
3,smallCI_smallSample,0.000,0.210,0.789
